In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN2004"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../../../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10


wavelet_levels = 6    #number of wavelet layer to have in the network
convolution_levels = 5    #number of convolution to have in the network
pooling_stride = [4, 2, 2, 2, 1]    #stride of pooling in convolution
weights_shape = [[5, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
granularity = np.prod(pooling_stride)
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [7]:
print pos_weight

3.18601822124


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [10]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [11]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation)
                        

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size)

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step = 100,
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.283084, fmeasure_max= 0.453357, precision= 0.308789, recall= 0.852459
Train Random guess:  , auc= 0.251434fmeasure_max= 0.385496, precision= 0.239148, recall= 0.993443
Validation Iter 0, auc= 0.237860, fmeasure_max= 0.412603, precision= 0.264042, recall= 0.943396
Validation Random guess:  auc= 0.232191, fmeasure_max= 0.372337, precision= 0.230146, recall= 0.974271
Minibatch Loss= 1.060083
Tarin Iter 100, auc= 0.607372, fmeasure_max= 0.578085, precision= 0.579585, recall= 0.576592
Train Random guess:  , auc= 0.225964fmeasure_max= 0.370064, precision= 0.227042, recall= 1.000000
Validation Iter 100, auc= 0.609077, fmeasure_max= 0.622307, precision= 0.493467, recall= 0.842196
Validation Random guess:  auc= 0.253866, fmeasure_max= 0.371630, precision= 0.228583, recall= 0.993139
Minibatch Loss= 0.837488
Tarin Iter 200, auc= 0.740680, fmeasure_max= 0.713306, precision= 0.636475, recall= 0.811232
Train Random guess:  , auc= 0.241698fmeasure_max= 0.400500, precision= 0.2503

In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [14]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.941373 F1_max = 0.866212 prec = 0.863522 rec = 0.868919
Test Random guess: auc= 0.231167 fmeasure_max= 0.375658 , precision= 0.231268 , recall= 1.000000


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})